# Part 2 project: Evaluating a decision tree

In [ ]:
import numpy as np
import sklearn.datasets
import sklearn.tree
import matplotlib
import matplotlib.pyplot as plt

from IPython.display import Image, YouTubeVideo

Array-oriented programming is usually concerned with number-crunching, such as simulations or data analysis. Usually not data structures. However, graphs and trees can be expressed in terms of arrays.

In this exercise, we'll take a decision tree built by Scikit-Learn and traverse it in an array-oriented way. It may seem that this isn't a good problem for array-oriented programming because it has to "iterate until converged" (walk down the tree until you reach a leaf node), but other advantages outweigh it.

## Obtaining a decision tree

One way to get a tree is to give Scikit-Learn's decision tree model a classification problem.

In [ ]:
X1, y1 = sklearn.datasets.make_gaussian_quantiles(
    cov=2.0, n_samples=500, n_features=2, n_classes=2, random_state=1
)
X2, y2 = sklearn.datasets.make_gaussian_quantiles(
    mean=(3, 3), cov=1.5, n_samples=1000, n_features=2, n_classes=2, random_state=1
)

X = np.concatenate((X1, X2))
y = np.concatenate((y1, -y2 + 1))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

ax.scatter(X[y == 0, 0], X[y == 0, 1], c="deepskyblue", edgecolor="black");
ax.scatter(X[y == 1, 0], X[y == 1, 1], c="orange", edgecolor="black");

The objective is to find a function of _x_ and _y_ that predicts whether a dot will be orange or blue.

In [ ]:
decision_tree = sklearn.tree.DecisionTreeClassifier(max_depth=10)
decision_tree.fit(X, y)

The function of _x_ and _y_ colors the plane below. The original points are overlaid with transparency.

(Standard) decision trees are restricted to horizontal and vertical cuts. This one is overfitted, as the region between the two blobs is finely divided into horizontal and vertical bands meant to catch a few of the overlapping points.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

xx, yy = np.meshgrid(np.arange(-5, 8, 0.02), np.arange(-5, 8, 0.02))
Z = decision_tree.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

ax.contourf(xx, yy, Z);
ax.scatter(X[y == 0, 0], X[y == 0, 1], c="deepskyblue", edgecolor="black", alpha=0.3);
ax.scatter(X[y == 1, 0], X[y == 1, 1], c="orange", edgecolor="black", alpha=0.3);
ax.set_xlim(-5, 8);
ax.set_ylim(-5, 8);

It's not important whether this is a good fit (it's not); what's important is that we now have a tree to play with.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

sklearn.tree.plot_tree(decision_tree, feature_names=["x", "y"], filled=True, ax=ax);

<br><br><br>

## Representing trees

In computer science classes, binary trees are usually presented as objects with pointers to two children, like this.

In [ ]:
from dataclasses import dataclass
from typing import Optional

@dataclass
class Node:
    feature: str
    threshold: float
    left: Optional["Node"]
    right: Optional["Node"]
    winner: str

    def predict(self, x: float, y: float) -> str:
        if self.left is None and self.right is None:
            return self.winner
        
        elif self.left is None:
            return self.right.predict(x, y)
        
        elif self.right is None:
            return self.left.predict(x, y)
        
        else:
            if self.feature == "x":
                value = x
            else:
                value = y

            if value <= self.threshold:
                return self.left.predict(x, y)
            else:
                return self.right.predict(x, y)

Here is a small instance of that kind of tree. It has 3 nodes.

In [ ]:
tree = Node(
    "x",
    3.14,
    Node("y", 2.71, None, None, "orange"),
    Node("x", 1.62, None, None, "blue"),
    "neither",
)

This type of tree has a `predict` method and enough attributes on each node to decide, for a given `x` and `y`, whether to follow its `left` child or its `right` child.

In [ ]:
tree.predict(1, 2)

In [ ]:
tree.predict(4, 1)

<br><br><br>

It is a flow chart.

<img src="../img/flowchart.png" width="50%">

<br><br><br>

However, Scikit-Learn stores its trees as arrays.

In [ ]:
decision_tree.tree_.children_left

In [ ]:
decision_tree.tree_.children_right

In [ ]:
decision_tree.tree_.feature

In [ ]:
decision_tree.tree_.threshold

<br><br><br>

We'll see in a moment how `decision_tree.tree_.children_left` and `children_right` represent the tree structure, but the other arrays are node attributes, like our `Node.feature` and `Node.threshold`. Each element in the arrays is either a tree node or a placeholder (for nodes without children).

<br><br><br>

The array values in `decision_tree.tree_.children_left` and `children_right` are like pointers in the C language, but instead of pointing to raw memory addresses, they point to other indexes in the same array.

`-1` is a placeholder for "no node" (like C's `NULL` pointer).

To demonstrate, let's try walking along these trees, starting at index `0`.

In [ ]:
def traverse_always_right(index):
    print(index)
    if index >= 0:
        traverse_always_right(decision_tree.tree_.children_right[index])

traverse_always_right(0)

Or zig-zag: left, right, left, right, ...

In [ ]:
def traverse_left_right_zigzag(index, which_way):
    print(which_way, index)
    if index >= 0:
        if which_way == "left ":
            traverse_left_right_zigzag(decision_tree.tree_.children_left[index], "right")
        else:
            traverse_left_right_zigzag(decision_tree.tree_.children_right[index], "left ")

traverse_left_right_zigzag(0, "left ")

So for this tree representation, array `__getitem__` means "tree traversal."

<br><br><br>

To check this interpretation, let's write a tree-walking function and compre it to Scikit-Learn's.

In [ ]:
def print_tree(tree, array_index=0, indent="", feature_names=["x", "y"]):
    has_children = tree.children_left[array_index] >= 0

    if has_children:
        feature = tree.feature[array_index]
        threshold = tree.threshold[array_index]
        left_index = tree.children_left[array_index]
        right_index = tree.children_right[array_index]

        yield f"{indent}{feature_names[feature]} <= {threshold:.2f}"
        yield from print_tree(tree, left_index, indent + "    ", feature_names)

        yield f"{indent}{feature_names[feature]} > {threshold:.2f}"
        yield from print_tree(tree, right_index, indent + "    ", feature_names)

    else:
        # tree.value is a count of the number of training data of each class that would reach this node
        # the largest number is what this tree predicts
        winner = np.argmax(tree.value[array_index])

        yield f"{indent}class: {winner}"

In [ ]:
for mine, theirs in zip(
    print_tree(decision_tree.tree_),
    sklearn.tree.export_text(decision_tree, feature_names=["x", "y"]).split("\n"),
):
    print(f"{mine:60s} {theirs:60s}")

They're the same!

<br><br><br>

## Single tree traversal

Scikit-Learn's `decision_tree.predict` function could be implemented for a single `x`, `y` position by modifying the above function, to walk the tree and report the final leaf node's class.

In [ ]:
def predict_single(position, tree, array_index=0):
    has_children = tree.children_left[array_index] >= 0

    if has_children:
        feature = tree.feature[array_index]
        threshold = tree.threshold[array_index]
        left_index = tree.children_left[array_index]
        right_index = tree.children_right[array_index]

        if position[feature] <= threshold:
            return predict_single(position, tree, left_index)
        else:
            return predict_single(position, tree, right_index)

    else:
        return np.argmax(tree.value[array_index], axis=1)

To see that this function works, we can make the prediction/training data overlay again, but using our function instead of Scikit-Learn's.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

xx, yy = np.meshgrid(np.arange(-5, 8, 0.02), np.arange(-5, 8, 0.02))

Z = np.array(
    [predict_single(position, decision_tree.tree_) for position in np.c_[xx.ravel(), yy.ravel()]]
).reshape(xx.shape)

ax.contourf(xx, yy, Z);
ax.scatter(X[y == 0, 0], X[y == 0, 1], c="deepskyblue", edgecolor="black", alpha=0.3);
ax.scatter(X[y == 1, 0], X[y == 1, 1], c="orange", edgecolor="black", alpha=0.3);
ax.set_xlim(-5, 8);
ax.set_ylim(-5, 8);

It's correct but slow. One reason is that our function is implementented in pure Python, but another is that each walk down the tree is a separate traversal.

On the plane above, each `x`, `y` point initiated another traversal down the same tree. What if we could traverse that tree in a vectorized way over all `x`, `y` points?

<br><br><br>

## THE EXERCISE

The goal of this exercise is to replace this:

In [ ]:
Image(filename="../img/plinko-price-is-right.gif") 

with this:

In [ ]:
YouTubeVideo('AuEUAXlbE94', width=600, height=450)

<br><br><br>

There's only one tree, but many `x`, `y` points to test.

Remember that slicing an array with an array of integer indexes returns an array of selected items:

In [ ]:
array = np.array([0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])

In [ ]:
indexes = np.array([4, 3, 3, 1, 6, 7, 8])

In [ ]:
array[indexes]

as though we can passed each integer index to `array.__getitem__` individually:

In [ ]:
for i in indexes:
    print(array[i])

So, for instance,

In [ ]:
indexes = np.array([0, 1, 2, 4, 6, 7, 8, 10])

decision_tree.tree_.threshold[indexes]

<br><br><br>

 * There are ways of solving this problem that keep all arrays immutable ("construct and read from arrays, but don't write to them after making them").
 * There are ways to do it that change the arrays after construction.

Either of these is acceptable: just produce the same output as running `predict_single` on every element of `prepared_data`.

<br><br><br>

The interface is

In [ ]:
def predict_many(positions, tree):
    ...

where `positions` is the `prepared_data` and `tree` is the `decision_tree.tree_`.

<br><br><br>

Note that the tree has a maximum depth.

In [ ]:
decision_tree.tree_.max_depth

<br><br><br>

**Suggestion:** implement

In [ ]:
def predict_many_step(positions, tree, array_indexes):
    ...

to see what one step of descending the tree does to an array of `array_indexes`. As with the Game of Life, you can repeatedly evaluate the cell to animate it.

<br><br><br>

**Question:** what do you do when some indexes reach leaf nodes and others don't?

<br><br><br>

<details>
    <summary><b>Hint!</b></summary>

Given an array of `positions` and `features = tree.feature[array_indexes]`, you can identify which ones will be taking the left child by

```python
choosing_left = positions[np.arange(len(positions)), features] <= thresholds
```

<br>

This is a 2-dimensional slice: along the first dimension of `positions` (the 422500 positions), we want every one, so we give it an array of indexes that are `[0, 1, 2, 2, ...]`. Along the second dimension, we pick `0` (_x_) or `1` (_y_), according to the values in the `features` array.

</details>

<br><br><br>

<details>
    <summary><b>Second hint!</b></summary>

The [np.where](https://numpy.org/doc/stable/reference/generated/numpy.where.html) function acts as an array-oriented if-then-else:

```python
np.where(choosing_left, left_indexes, right_indexes)
```

</details>

<br><br><br>

<details>
    <summary><b>Third hint!</b></summary>

A reasonable way to make indexes that have already reached leaf node $X$ stop iterating is to define their transition as $X \to X$. They're still "updated," but they're updated to the value they already have. Implementing this uses tools already described in the other hints (i.e. more of the same).

</details>